In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime as dt
import importlib

import aux_funcs as aux

In [2]:
test = aux.read_test_data()

test.csv read in 28.7 s
Mem. usage decreased to 596.49 Mb (53.1% reduction)
(41697600, 10)
   row_id  building_id  meter  timestamp  hour  day  year  weeknumber  \
0       0            0      0 2017-01-01     0    1  2017          52   
1       1            1      0 2017-01-01     0    1  2017          52   
2       2            2      0 2017-01-01     0    1  2017          52   
3       3            3      0 2017-01-01     0    1  2017          52   
4       4            4      0 2017-01-01     0    1  2017          52   

   weekday  month  
0        6      1  
1        6      1  
2        6      1  
3        6      1  
4        6      1  


In [3]:
weather_full_test = aux.read_full_weather_test_data()

Weather_test_fulldata.csv read in 0.3 s
Mem. usage decreased to  2.94 Mb (54.2% reduction)


In [4]:
building = aux.read_building_data()

building_metadata.csv read in 0.2 s
Mem. usage decreased to  0.03 Mb (60.3% reduction)
(1449, 7)
   site_id  building_id primary_use  square_feet  year_built  floor_count  \
0        0            0   Education         7432      2008.0          NaN   
1        0            1   Education         2720      2004.0          NaN   
2        0            2   Education         5376      1991.0          NaN   
3        0            3   Education        23685      2002.0          NaN   
4        0            4   Education       116607      1975.0          NaN   

   log_square_feet  
0         8.913550  
1         7.908387  
2         8.589700  
3        10.072597  
4        11.666565  


### join

In [5]:
#merge tables
testbw = test.join(building, on='building_id', rsuffix = 'r')
testbw.drop(columns=['building_idr'],inplace=True)
print(testbw.shape)
testbw.head()

(41697600, 16)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,year_built,floor_count,log_square_feet
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,2008.0,NaN,8.913550
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,2004.0,NaN,7.908387
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,1991.0,NaN,8.589700
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,2002.0,NaN,10.072597
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,1975.0,NaN,11.666565


In [6]:
testbw = pd.merge(testbw, weather_full_test,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')
print(testbw.shape)
testbw.head()

(41697600, 17)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,year_built,floor_count,log_square_feet,air_temperature
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,2008.0,NaN,8.913550,17.796875
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,2004.0,NaN,7.908387,17.796875
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,1991.0,NaN,8.589700,17.796875
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,2002.0,NaN,10.072597,17.796875
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,1975.0,NaN,11.666565,17.796875


In [7]:
from sklearn.tree import DecisionTreeRegressor
# read trees
import pickle
with open('tuned_tree_of_trees_191026_v02.pkl','rb') as file:
    dtrs = pickle.load(file)

In [9]:
testbw['log_meter_reading'] =0

In [10]:
features_list=['weeknumber','weekday','hour','log_square_feet','air_temperature']

In [11]:
subgroups = testbw.groupby(['primary_use','meter'])['meter'].count()
subgroups

primary_use                    meter
Education                      0        9408240
                               1        3784320
                               2        2382720
                               3        1226400
Entertainment/public assembly  0        3136080
                               1         770880
                               2         525600
                               3         280320
Food sales and service         0          87600
                               1          70080
                               2          35040
                               3          35040
Healthcare                     0         367920
                               1         227760
                               2         157680
                               3          52560
Lodging/residential            0        2540400
                               1         981120
                               2         788400
                               3         192720
Man

In [12]:
subix = subgroups.index
for k in range(0, len(subgroups)):

    use = subix[k][0]
    meter = subix[k][1]
    
    print('----------------')
    print(k, use, meter)
    print('   Filter by use and meter...')
    # condition
    cond = (testbw['primary_use']== use) & (testbw['meter']==meter)
    # select features
    print('   Select features and datapoints...')
    X = testbw.loc[cond, features_list]
    #pick tree
    dtr = dtrs[k]
    
    # predict
    print('   Predicting...')
    testbw.loc[cond,'log_meter_reading'] = dtr.predict(X)
    print('   Done!')
    print('')

----------------
0 Education 0
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
1 Education 1
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
2 Education 2
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
3 Education 3
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
4 Entertainment/public assembly 0
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
5 Entertainment/public assembly 1
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
6 Entertainment/public assembly 2
   Filter by use and meter...
   Select features and datapoints...
   Predicting...
   Done!

----------------
7 Entertainment/public assembly 3
   Filter by u

In [13]:
# convert from log to linear
testbw['meter_reading'] = np.exp(testbw['log_meter_reading'])-1

In [14]:
testbw.head()

,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,year_built,floor_count,log_square_feet,air_temperature,log_meter_reading,meter_reading
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,2008.0,NaN,8.913550,17.796875,5.548921,255.960052
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,2004.0,NaN,7.908387,17.796875,4.003429,53.785711
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,1991.0,NaN,8.589700,17.796875,1.777390,4.914400
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,2002.0,NaN,10.072597,17.796875,5.286129,196.577028
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,1975.0,NaN,11.666565,17.796875,7.028724,1127.589829


In [15]:
#export results
filename = 'submission_02_191026.csv'
# 
testbw.loc[:,['row_id','meter_reading']].to_csv(filename,index=False)

In [16]:
aux.get_lines(filename)

['row_id,meter_reading\n',
 '0,255.96005155910484\n',
 '1,53.78571105827696\n',
 '2,4.914400397169019\n',
 '3,196.57702750026465\n']